Nonlinear Elasticity
===

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

importing NGSolve-6.2.2204


A rectangular mesh:

In [2]:
def GenerateMesh(t=0.1):
    bar = MoveTo(0,-t/2).Rectangle(1,t).Face()
    bar.edges.Min(X).name="left"
    bar.edges.Max(X).name="right"
    bar.edges.Min(X).maxh=t/8
    mesh = Mesh(OCCGeometry(bar, dim=2).GenerateMesh(maxh=t/2.5))
    return mesh
mesh = GenerateMesh()
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

Cauchy-Green tensor and hyperelastic energy density: 
\begin{align}
\Psi^{H}(E) &= \mu\,\|\varepsilon(u)\|^2+\frac{\lambda}{2}\mathrm{tr}(\varepsilon(u))^2\\
\Psi^{VK}(E) &= \mu\,\|E\|^2+\frac{\lambda}{2}\mathrm{tr}(E)^2\\
\Psi^{NH}(C) &= \frac{\mu}{2}(\mathrm{tr}(C-I)-\log(\det(C))) + \frac{\lambda}{2}(\sqrt{\det(C)}-1)^2
\end{align}

In [3]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def C(u):
    F = Id(2) + Grad(u)
    return F.trans * F

def Hooke(u):
    E = Sym(Grad(u))
    return mu*InnerProduct(E,E) + lam/2*Trace(E)**2

def StVenantKirchhoff(u):
    E = 0.5*(C(u)-Id(2))
    return mu*InnerProduct(E,E) + lam/2*Trace(E)**2

def NeoHooke(u):
    #return mu/2*(Trace(C(u)-Id(2))- log(Det(C(u)))) + lam/8*(log(Det(C(u))))**2
    return mu/2*(Trace(C(u)-Id(2))- log(Det(C(u)))) + lam/2*(Det(Id(2)+Grad(u))-1)**2

Energy minimization formulation
$$\mathcal{W}(u) = \int_{\Omega} W(C(u)) - f u\,dx \to\min!$$

In [4]:
factor = Parameter(0)
force = CF( (0,factor) )

fes = VectorH1(mesh, order=3, dirichlet="left")
u  = fes.TrialFunction()

aNH = BilinearForm(fes, symmetric=True)
aNH += Variation(NeoHooke(u).Compile()*dx)
aNH += Variation((-force*u).Compile()*dx)

gfuNH = GridFunction(fes)

a simple Newton solver, using automatic differentiation for residual and tangential stiffness:

In [5]:
def SolveNewton():
    res = gfuNH.vec.CreateVector()
    
    for it in range(10):
        print ("it", it, "energy = ", aNH.Energy(gfuNH.vec))
        aNH.Apply(gfuNH.vec, res)
        aNH.AssembleLinearization(gfuNH.vec)
        inv = aNH.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky" ) 
        gfuNH.vec.data -= inv*res

In [6]:
factor.Set(0.4)
SolveNewton()
scene = Draw (C(gfuNH)[0,0], mesh, deformation=gfuNH)

it 0 energy =  0.0
it 1 energy =  0.06480141734179994
it 2 energy =  -0.0021435227242957724
it 3 energy =  -0.0021708981748938517
it 4 energy =  -0.0021709019127899615
it 5 energy =  -0.002170901912876264
it 6 energy =  -0.0021709019128762406
it 7 energy =  -0.0021709019128762354
it 8 energy =  -0.002170901912876221
it 9 energy =  -0.0021709019128762375


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

In [12]:
factor.Set(factor.Get()+0.4)
solvers.Newton(aNH,gfuNH, inverse="sparsecholesky")
scene.Redraw()

Newton iteration  0
err =  0.010077704137454057
Newton iteration  1
err =  0.0003945133896852711
Newton iteration  2
err =  6.746492908325972e-07
Newton iteration  3
err =  2.8677800397136756e-12


## Compare linear Hooke, St. Venant-Kirchhoff and Neo-Hooke

In [7]:
def Solve(mesh, order=2):
    fes = VectorH1(mesh, order=order, dirichlet="left")
    u,v = fes.TnT()

    a = BilinearForm(fes, symmetric=True)
    a += Variation( Hooke(u)*dx-force*u*dx)

    gfu = GridFunction(fes)
    
    r = gfu.vec.CreateVector()
    a.Apply(gfu.vec, r)
    a.AssembleLinearization(gfu.vec)

    gfu.vec.data -= a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*r
    return gfu

In [8]:
factor.Set(8)
gfu = Solve(mesh)
Draw(Det(Id(2)+Grad(gfu)), mesh, deformation=gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [9]:
aVK = BilinearForm(fes, symmetric=True)
aVK += Variation((StVenantKirchhoff(u)-force*u).Compile()*dx)

gfuVK = GridFunction(fes)

scene = Draw (Det(Id(2)+Grad(gfuVK)), mesh, deformation=gfuVK)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

In [10]:
numsteps=10
with TaskManager():
    for i in range(numsteps):
        factor.Set(5*(i+1)/numsteps)
        print("loadstep = ", 5*(i+1)/numsteps)
        (result,it) = solvers.Newton(aVK, gfuVK, maxerr=1e-9, printing=False, maxit=50, inverse="sparsecholesky")
        if result == 0:
            scene.Redraw()
        else:
            break

loadstep =  0.5
loadstep =  1.0
loadstep =  1.5
loadstep =  2.0
loadstep =  2.5
loadstep =  3.0
loadstep =  3.5
loadstep =  4.0


In [11]:
gfuNH.vec[:] = 0
scene = Draw (Det(Id(2)+Grad(gfuNH)), mesh, deformation=gfuNH)
numsteps=10
with TaskManager():
    for i in range(numsteps):
        factor.Set(10*(i+1)/numsteps)
        print("loadstep = ", 10*(i+1)/numsteps)
        (result,it) = solvers.Newton(aNH, gfuNH, maxerr=1e-9, printing=False, maxit=30, inverse="sparsecholesky")
        if result == 0:
            scene.Redraw()
        else:
            break

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

loadstep =  1.0
loadstep =  2.0
loadstep =  3.0
loadstep =  4.0
loadstep =  5.0
loadstep =  6.0
loadstep =  7.0
loadstep =  8.0
loadstep =  9.0
loadstep =  10.0
